In [67]:
%run ../0-utils/0-Base.ipynb

Read data about customers (`train` and `test`):

In [68]:
customers_dfs = dict()

for clazz in ("train", "test"):
    df = pd.read_csv(f"../data/raw/{clazz}.csv", parse_dates=["first_active_month"])

    # test.csv has one missing value - replace it with the most popular 
    # first_active_month for rows in train.csv that have the same feature_x
    if "test" == clazz: df.fillna(pd.to_datetime("2017-01"), inplace=True)
        
    if "target" not in df.columns: df["target"] = 0

    feature_1_backup = df.feature_1
    feature_2_backup = df.feature_2

    df = pd.get_dummies(df, columns=["feature_1", "feature_2"])

    df["feature_1"] = feature_1_backup
    df["feature_2"] = feature_2_backup
    
    display(df)
    
    customers_dfs[clazz] = df

,first_active_month,card_id,feature_3,target,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_1,feature_2
0,2017-06-01,C_ID_92a2005557,1,-0.820283,0,0,0,0,1,0,1,0,5,2
1,2017-01-01,C_ID_3d0044924f,0,0.392913,0,0,0,1,0,1,0,0,4,1
2,2016-08-01,C_ID_d639edf6cd,0,0.688056,0,1,0,0,0,0,1,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201914,2017-08-01,C_ID_7666735b3d,0,0.093494,0,0,0,1,0,0,0,1,4,3
201915,2016-07-01,C_ID_73f5a0efd0,1,-4.676589,0,0,1,0,0,0,1,0,3,2
201916,2017-07-01,C_ID_92c9984c58,1,-1.859413,0,0,1,0,0,1,0,0,3,1


,first_active_month,card_id,feature_3,target,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_1,feature_2
0,2017-04-01,C_ID_0ab67a22ab,1,0,0,0,1,0,0,0,0,1,3,3
1,2017-01-01,C_ID_130fd0cbdd,0,0,0,1,0,0,0,0,0,1,2,3
2,2017-08-01,C_ID_b709037bc5,1,0,0,0,0,0,1,1,0,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123620,2016-09-01,C_ID_21d56d950c,1,0,0,0,0,0,1,1,0,0,5,1
123621,2017-06-01,C_ID_6c46fc5a9d,0,0,0,1,0,0,0,1,0,0,2,1
123622,2016-10-01,C_ID_87e7979a5f,1,0,0,0,0,0,1,1,0,0,5,1


Define a sample partition of customers:

In [69]:
sample_customers_df = add_part(customers_dfs["train"], n_parts=TRANSACTIONS_N_PARTS["new"])
sample_customers_df = sample_customers_df.loc[lambda x: x.part == 13].drop("part", axis=1)

display(sample_customers_df)

,first_active_month,card_id,feature_3,target,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_1,feature_2
43,2017-11-01,C_ID_1919f9080e,0,0.393388,0,1,0,0,0,0,0,1,2,3
225,2017-01-01,C_ID_ae77d244b6,0,-33.219281,0,1,0,0,0,1,0,0,2,1
243,2017-09-01,C_ID_ee5bb1f392,0,-1.230539,0,1,0,0,0,0,0,1,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201682,2017-08-01,C_ID_5ac91f1427,0,-2.315998,0,1,0,0,0,1,0,0,2,1
201899,2017-10-01,C_ID_58e359763e,1,-2.702214,0,0,0,0,1,1,0,0,5,1
201910,2017-10-01,C_ID_9072609210,0,-1.753271,0,1,0,0,0,0,0,1,2,3


Define a function for reading and processing a partition with transactions:

In [70]:
from datetime import datetime
from pandas.api.types import CategoricalDtype

transactions_cat_cols = ("city_id",
                         "category_3",
                         "merchant_category_id",
                         "state_id",
                         "subsector_id")

transactions_dtype = {cat_col: "category" for cat_col in transactions_cat_cols}

transactions_dtype["installments"] = CategoricalDtype(
    categories=np.arange(-1, 13).tolist() + [999],
    ordered=True,
)

def read_transactions_part(part_file_name):
    transactions_df = pd.read_csv(f"../data/1-partitioned/{part_file_name}",
                                  parse_dates=["purchase_date"],
                                  dtype=transactions_dtype)
    
    transactions_df = transactions_df.reset_index().rename(columns={"index": "ID"})

    days_diff = datetime.today() - transactions_df.purchase_date
    transactions_df["month_diff"] = days_diff // np.timedelta64(1, "M")
    transactions_df["month_diff"] += transactions_df.month_lag
    
    cat_cols_mapping = dict(
        authorized_flag=dict(
            Y=1,
            N=0,
        ),
        category_1=dict(
            Y=1,
            N=0,
        ),
        category_3=dict(
            A=0,
            B=1,
            C=2,
        ),
    )
    
    for col, mapping in cat_cols_mapping.items():
        transactions_df[col] = transactions_df[col].map(mapping)
    
    for col in ("authorized_flag", "category_1", "category_2", "category_3"):
        transactions_df[col] = transactions_df[col].fillna(-1).astype(int)
        
    category_2_backup = transactions_df.category_2
    category_3_backup = transactions_df.category_3
    installments_backup = transactions_df.installments

    transactions_df = pd.get_dummies(transactions_df, columns=["category_2", "category_3", "installments"])
    
    transactions_df["category_2"] = category_2_backup
    transactions_df["category_3"] = category_3_backup
    transactions_df["installments"] = installments_backup

    return transactions_df

Read a sample partition with `read_transactions_part(...)`:

In [71]:
%%time

sample_transactions_df = read_transactions_part("new.013.csv")

display(sample_transactions_df)

,ID,authorized_flag,card_id,city_id,category_1,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,month_diff,category_2_-1,category_2_1,category_2_2,category_2_3,category_2_4,category_2_5,category_3_-1,category_3_0,category_3_1,category_3_2,installments_-1,installments_0,installments_1,installments_2,installments_3,installments_4,installments_5,installments_6,installments_7,installments_8,installments_9,installments_10,installments_11,installments_12,installments_999,category_2,category_3,installments
0,0,1,C_ID_be71a60d23,143,0,19,M_ID_ab392e3714,1,-0.701753,2018-03-01 05:47:19,5,36,12,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0
1,1,1,C_ID_be71a60d23,333,0,307,M_ID_1fb0d13ea0,1,-0.716855,2018-03-08 18:27:49,9,19,12,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2,1,C_ID_be71a60d23,143,0,166,M_ID_12c118b802,2,-0.736389,2018-04-30 17:44:44,5,29,11,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20756,20756,1,C_ID_7750eb98a6,20,0,818,M_ID_1b5a3c1373,2,2.825675,2018-04-19 12:02:33,19,12,11,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,1,1
20757,20757,1,C_ID_438aea7835,53,0,557,M_ID_8013941341,1,-0.665765,2018-03-28 08:57:19,20,29,11,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0
20758,20758,1,C_ID_611d4d588f,302,0,307,M_ID_ee5a6f6a35,2,-0.731881,2018-03-11 15:07:49,7,19,12,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,1,1


CPU times: user 136 ms, sys: 0 ns, total: 136 ms
Wall time: 135 ms


In [72]:
%%time

merchants_df = pd.read_csv("../data/raw/merchants.csv")
merchants_df = merchants_df.drop_duplicates("merchant_id")
merchants_df = merchants_df.reset_index().rename(columns={"index": "ID"})

cat_cols_mapping = dict(
    category_1=dict(
        Y=1,
        N=0,
    ),
    category_4=dict(
        Y=1,
        N=0,
    ),
    most_recent_sales_range=dict(
        A=0,
        B=1,
        C=2,
        D=3,
        E=4,
    ),
    most_recent_purchases_range=dict(
        A=0,
        B=1,
        C=2,
        D=3,
        E=4,
    ),
)

for col, mapping in cat_cols_mapping.items():
    merchants_df[col] = merchants_df[col].map(mapping)

for col in ("category_1", "category_2", "category_4", "most_recent_sales_range", "most_recent_purchases_range"):
    merchants_df[col] = merchants_df[col].fillna(-1).astype(int)

category_2_backup = merchants_df.category_2
most_recent_sales_range_backup = merchants_df.most_recent_sales_range
most_recent_purchases_range_backup = merchants_df.most_recent_purchases_range

merchants_df = pd.get_dummies(merchants_df, columns=["category_2",
                                                     "most_recent_sales_range",
                                                     "most_recent_purchases_range"])

merchants_df["category_2"] = category_2_backup
merchants_df["most_recent_sales_range"] = most_recent_sales_range_backup
merchants_df["most_recent_purchases_range"] = most_recent_purchases_range_backup

merchants_df = merchants_df.fillna(0)

display(merchants_df)

,ID,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2_-1,category_2_1,category_2_2,category_2_3,category_2_4,category_2_5,most_recent_sales_range_0,most_recent_sales_range_1,most_recent_sales_range_2,most_recent_sales_range_3,most_recent_sales_range_4,most_recent_purchases_range_0,most_recent_purchases_range_1,most_recent_purchases_range_2,most_recent_purchases_range_3,most_recent_purchases_range_4,category_2,most_recent_sales_range,most_recent_purchases_range
0,0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,0,-0.40,9.666667,3,-2.25,18.666667,6,-2.32,13.916667,12,0,242,9,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,4,4
1,1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,0,-0.72,1.750000,3,-0.74,1.291667,6,-0.57,1.687500,12,0,22,16,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,4,4
2,2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,0,-82.13,260.000000,2,-82.13,260.000000,2,-82.13,260.000000,2,0,-1,5,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,5,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334630,334693,M_ID_f2045dd267,35,561,7,-0.057471,-0.057471,0,0.96,0.982957,3,0.90,0.924769,6,0.74,0.750763,8,1,160,21,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,5,0,0
334631,334694,M_ID_9139332ccc,35,511,7,-0.057471,-0.057471,1,0.94,0.919558,3,0.82,0.783000,6,0.65,0.584000,12,1,-1,-1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,-1,0,0
334632,334695,M_ID_cd2c0b07e9,35,606,17,-0.057471,-0.057471,0,0.90,0.913902,3,0.73,0.744417,6,0.53,0.540334,10,1,69,9,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0


CPU times: user 1.08 s, sys: 31.9 ms, total: 1.12 s
Wall time: 1.11 s


Define a function for building an entry set for `customers_df` and `transactions_df`:

In [73]:
import featuretools as ft
import featuretools.variable_types as vtypes

transactions_vtypes = dict(authorized_flag=vtypes.Boolean,
                           category_1=vtypes.Boolean)

for ord_col in ("installments", "category_2", "category_3"):
    transactions_vtypes[ord_col] = vtypes.Ordinal
    
    for bool_col in filter(lambda x: x.startswith(ord_col + "_"), sample_transactions_df.columns):
        transactions_vtypes[bool_col] = vtypes.Boolean
        
customers_vtypes = {
    "feature_1": vtypes.Ordinal,
    "feature_2": vtypes.Ordinal,
    "feature_3": vtypes.Boolean,
}

merchants_vtypes = dict(category_1=vtypes.Boolean,
                        category_4=vtypes.Boolean)

for ord_col in ("category_2", "most_recent_sales_range", "most_recent_purchases_range"):
    merchants_vtypes[ord_col] = vtypes.Ordinal
    
    for bool_col in filter(lambda x: x.startswith(ord_col + "_"), merchants_df.columns):
        merchants_vtypes[bool_col] = vtypes.Boolean

def build_entity_set(customers_df, transactions_df, merchants_df):
    es = ft.EntitySet()
    
    es.entity_from_dataframe(entity_id="customers",
                             dataframe=customers_df,
                             index="card_id",
                             variable_types=customers_vtypes)

    es.entity_from_dataframe(entity_id="transactions",
                             dataframe=transactions_df,
                             index="ID",
                             variable_types=transactions_vtypes)
    
    es.entity_from_dataframe(entity_id="merchants",
                             dataframe=merchants_df,
                             index="merchant_id",
                             variable_types=merchants_vtypes)

    customer_transactions_relationship = ft.Relationship(es["customers"]["card_id"],
                                                         es["transactions"]["card_id"])
    
    merchants_transactions_relationship = ft.Relationship(es["merchants"]["merchant_id"],
                                                          es["transactions"]["merchant_id"])

    es.add_relationships([customer_transactions_relationship,
                          merchants_transactions_relationship])

    es["transactions"]["category_1"].interesting_values = [1]
    es["transactions"]["authorized_flag"].interesting_values = [1]

    for col in ("category_2", "category_3", "installments"):
        for dummy_col in filter(lambda x: x.startswith(col + "_"), transactions_df.columns):
            es["transactions"][dummy_col].interesting_values = [1]
            
    es["merchants"]["category_1"].interesting_values = [1]
    es["merchants"]["category_4"].interesting_values = [1]
    
    for col in ("category_2", "most_recent_sales_range", "most_recent_purchases_range"):
        for dummy_col in filter(lambda x: x.startswith(col + "_"), merchants_df.columns):
            es["merchants"][dummy_col].interesting_values = [1]

    return es

Build an entity set for sample data:

In [74]:
es = build_entity_set(sample_customers_df, sample_transactions_df, merchants_df)

Do deep feature synthesis:

In [75]:
features = ft.dfs(entityset=es, target_entity="customers", features_only=True)

print("Number of features: %d" % len(features))

display(features)

Number of features: 221


[<Feature: target>,
 <Feature: feature_1_1>,
 <Feature: feature_1_2>,
 <Feature: feature_1_3>,
 <Feature: feature_1_4>,
 <Feature: feature_1_5>,
 <Feature: feature_2_1>,
 <Feature: feature_2_2>,
 <Feature: feature_2_3>,
 <Feature: feature_1>,
 <Feature: feature_2>,
 <Feature: feature_3>,
 <Feature: SUM(transactions.month_lag)>,
 <Feature: SUM(transactions.purchase_amount)>,
 <Feature: SUM(transactions.month_diff)>,
 <Feature: STD(transactions.month_lag)>,
 <Feature: STD(transactions.purchase_amount)>,
 <Feature: STD(transactions.month_diff)>,
 <Feature: MAX(transactions.month_lag)>,
 <Feature: MAX(transactions.purchase_amount)>,
 <Feature: MAX(transactions.month_diff)>,
 <Feature: SKEW(transactions.month_lag)>,
 <Feature: SKEW(transactions.purchase_amount)>,
 <Feature: SKEW(transactions.month_diff)>,
 <Feature: MIN(transactions.month_lag)>,
 <Feature: MIN(transactions.purchase_amount)>,
 <Feature: MIN(transactions.month_diff)>,
 <Feature: MEAN(transactions.month_lag)>,
 <Feature: MEAN(

In [76]:
feature_matrix_df = ft.calculate_feature_matrix(features=features, entityset=es, verbose=True)

display(feature_matrix_df)

Elapsed: 00:33 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 10/10 chunks


,target,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_1,feature_2,feature_3,SUM(transactions.month_lag),SUM(transactions.purchase_amount),SUM(transactions.month_diff),STD(transactions.month_lag),STD(transactions.purchase_amount),STD(transactions.month_diff),MAX(transactions.month_lag),MAX(transactions.purchase_amount),MAX(transactions.month_diff),SKEW(transactions.month_lag),SKEW(transactions.purchase_amount),SKEW(transactions.month_diff),MIN(transactions.month_lag),MIN(transactions.purchase_amount),MIN(transactions.month_diff),MEAN(transactions.month_lag),MEAN(transactions.purchase_amount),MEAN(transactions.month_diff),COUNT(transactions),PERCENT_TRUE(transactions.authorized_flag),PERCENT_TRUE(transactions.category_1),PERCENT_TRUE(transactions.installments_-1),PERCENT_TRUE(transactions.installments_0),PERCENT_TRUE(transactions.installments_1),PERCENT_TRUE(transactions.installments_2),PERCENT_TRUE(transactions.installments_3),PERCENT_TRUE(transactions.installments_4),PERCENT_TRUE(transactions.installments_5),PERCENT_TRUE(transactions.installments_6),PERCENT_TRUE(transactions.installments_7),PERCENT_TRUE(transactions.installments_8),PERCENT_TRUE(transactions.installments_9),PERCENT_TRUE(transactions.installments_10),PERCENT_TRUE(transactions.installments_11),PERCENT_TRUE(transactions.installments_12),PERCENT_TRUE(transactions.installments_999),PERCENT_TRUE(transactions.category_2_-1),PERCENT_TRUE(transactions.category_2_1),PERCENT_TRUE(transactions.category_2_2),PERCENT_TRUE(transactions.category_2_3),PERCENT_TRUE(transactions.category_2_4),PERCENT_TRUE(transactions.category_2_5),PERCENT_TRUE(transactions.category_3_-1),PERCENT_TRUE(transactions.category_3_0),PERCENT_TRUE(transactions.category_3_1),PERCENT_TRUE(transactions.category_3_2),NUM_UNIQUE(transactions.city_id),NUM_UNIQUE(transactions.merchant_category_id),NUM_UNIQUE(transactions.merchant_id),NUM_UNIQUE(transactions.state_id),NUM_UNIQUE(transactions.subsector_id),NUM_UNIQUE(transactions.installments),NUM_UNIQUE(transactions.category_2),NUM_UNIQUE(transactions.category_3),MODE(transactions.city_id),MODE(transactions.merchant_category_id),MODE(transactions.merchant_id),MODE(transactions.state_id),MODE(transactions.subsector_id),MODE(transactions.installments),MODE(transactions.category_2),MODE(transactions.category_3),DAY(first_active_month),YEAR(first_active_month),MONTH(first_active_month),WEEKDAY(first_active_month),SUM(transactions.merchants.ID),SUM(transactions.merchants.merchant_group_id),SUM(transactions.merchants.merchant_category_id),SUM(transactions.merchants.subsector_id),SUM(transactions.merchants.numerical_1),SUM(transactions.merchants.numerical_2),SUM(transactions.merchants.avg_sales_lag3),SUM(transactions.merchants.avg_purchases_lag3),SUM(transactions.merchants.active_months_lag3),SUM(transactions.merchants.avg_sales_lag6),SUM(transactions.merchants.avg_purchases_lag6),SUM(transactions.merchants.active_months_lag6),SUM(transactions.merchants.avg_sales_lag12),SUM(transactions.merchants.avg_purchases_lag12),SUM(transactions.merchants.active_months_lag12),SUM(transactions.merchants.city_id),SUM(transactions.merchants.state_id),STD(transactions.merchants.ID),STD(transactions.merchants.merchant_group_id),STD(transactions.merchants.merchant_category_id),STD(transactions.merchants.subsector_id),STD(transactions.merchants.numerical_1),STD(transactions.merchants.numerical_2),STD(transactions.merchants.avg_sales_lag3),STD(transactions.merchants.avg_purchases_lag3),STD(transactions.merchants.active_months_lag3),STD(transactions.merchants.avg_sales_lag6),STD(transactions.merchants.avg_purchases_lag6),STD(transactions.merchants.active_months_lag6),STD(transactions.merchants.avg_sales_lag12),STD(transactions.merchants.avg_purchases_lag12),STD(transactions.merchants.active_months_lag12),STD(transactions.merchants.city_id),STD(transactions.merchants.state_id),MAX(transactions.merchants.ID),MAX(transactions.merchant

In [77]:
%%time

from multiprocessing import Pool

def calculate_feature_matrix_for(transactions_part_file_name, customers_part_df):
    transactions_part_df = read_transactions_part(transactions_part_file_name)

    part_entity_set = build_entity_set(
        customers_part_df,
        transactions_part_df,
        merchants_df,
    )
    
    feature_matrix_part_df = ft.calculate_feature_matrix(
        features=features, entityset=part_entity_set,
    )
    
    return feature_matrix_part_df

feature_matrix_dfs = dict()

for customers_clazz, customers_df in customers_dfs.items():
    for transactions_clazz, n_parts in TRANSACTIONS_N_PARTS.items():
        customers_with_part_df = add_part(customers_df, n_parts=n_parts)
        
        processes_args = []
        
        for part, customers_part_df in customers_with_part_df.groupby("part"):
            transactions_part_file_name = "%s.%03d.csv" % (transactions_clazz, part)
            processes_args.append((transactions_part_file_name, customers_part_df))
        
        with Pool(8) as p:
            feature_matrix_part_dfs = p.starmap(
                calculate_feature_matrix_for,
                processes_args,
            )

        feature_matrix_df = pd.concat(
            feature_matrix_part_dfs,
            sort=False,
        )
        
        feature_matrix_dfs[customers_clazz] = feature_matrix_df
        
        break # only old transactions for now

CPU times: user 4.3 s, sys: 996 ms, total: 5.3 s
Wall time: 36min 27s


In [78]:
%%time

for feature_matrix_clazz, feature_matrix_df in feature_matrix_dfs.items():
    feature_matrix_df.to_csv(f"../data/2-feature-engineered/{feature_matrix_clazz}.csv")

CPU times: user 57.3 s, sys: 355 ms, total: 57.6 s
Wall time: 57.6 s


In [15]:
context = ("display.max_rows",     0,
           "display.max_colwidth", 0,)

with pd.option_context(*context): display(ft.list_primitives())

,name,type,description
0,skew,aggregation,Finds the maximum non-null value of a numeric feature.
1,trend,aggregation,Finds the maximum non-null value of a numeric feature.
2,min,aggregation,Finds the maximum non-null value of a numeric feature.
3,avg_time_between,aggregation,Finds the maximum non-null value of a numeric feature.
4,median,aggregation,Finds the maximum non-null value of a numeric feature.
5,time_since_last,aggregation,Finds the maximum non-null value of a numeric feature.
6,n_most_common,aggregation,Finds the maximum non-null value of a numeric feature.
7,all,aggregation,Finds the maximum non-null value of a numeric feature.
8,mode,aggregation,Finds the maximum non-null value of a numeric feature.
9,percent_true,aggregation,Finds the maximum non-null value of a numeric feature.


Add a `fraud` boolean flag?

In [ ]:
authorized_card_rate = transactions_df.groupby("card_id").authorized_flag.mean().sort_values()

with pd.option_context("display.max_rows", 10): display(authorized_card_rate)

`999` installments looks like a `fraud` indicator too

In [ ]:
transactions_df.groupby("installments").authorized_flag.mean()

Use `np.log` for `purchase_amount`?

In [3]:
_, axs = plt.subplots(1, 2, figsize=(6, 3), sharey=True)
axs[0].hist(transactions_df.purchase_amount)
axs[1].hist(np.log(transactions_df.purchase_amount + 1))

plt.show()

NameError: name 'plt' is not defined